In [2]:
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_0.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_1.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_2.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_3.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_4.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_5.parquet
!wget https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_6.parquet    

--2023-03-26 23:29:29--  https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_0.parquet
Resolving cernbox.cern.ch (cernbox.cern.ch)... 137.138.120.151, 128.142.53.35, 128.142.53.28, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|137.138.120.151|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1171158319 (1.1G) [application/octet-stream]
Saving to: ‘top_gun_opendata_0.parquet’

top_gun_opendata_0. 100%[===================>]   1.09G  22.7MB/s    in 51s     

2023-03-26 23:30:31 (21.9 MB/s) - ‘top_gun_opendata_0.parquet’ saved [1171158319/1171158319]

--2023-03-26 23:30:32--  https://cernbox.cern.ch/remote.php/dav/public-files/F2rtz6k9TvaWynC/top_gun_opendata_1.parquet
Resolving cernbox.cern.ch (cernbox.cern.ch)... 128.142.53.28, 128.142.170.17, 128.142.53.35, ...
Connecting to cernbox.cern.ch (cernbox.cern.ch)|128.142.53.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1170397881 (1.1G) [application

In [4]:
from datasets import load_dataset
import cv2
from PIL import Image
from torch import nn
import torch
import pandas as pd
import os
import numpy as np
from datasets import load_metric
from sklearn.metrics import mean_squared_error
from transformers import BeitImageProcessor
from transformers import ViTFeatureExtractor
from torchvision import transforms
from transformers import  BeitForImageClassification, TrainingArguments, Trainer, AutoModelForImageClassification

In [2]:
data_files=[f"/kaggle/working/top_gun_opendata_{i}.parquet" for i in range(7)]
data_files

['/kaggle/working/top_gun_opendata_0.parquet',
 '/kaggle/working/top_gun_opendata_1.parquet',
 '/kaggle/working/top_gun_opendata_2.parquet',
 '/kaggle/working/top_gun_opendata_3.parquet',
 '/kaggle/working/top_gun_opendata_4.parquet',
 '/kaggle/working/top_gun_opendata_5.parquet',
 '/kaggle/working/top_gun_opendata_6.parquet']

## Code for loading dataset surely doesn't work
couldn't test it since my memory overflows then notebook crashes

In [ ]:
dataset_train = load_dataset("parquet", data_files=data_files)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset_train['train'][0]

In [ ]:
dataset_train['train'].features['Target']

In [6]:
model_id = 'google/vit-base-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(
  model_id,
  size={"height":125,"width":125},
  do_normalize=False
)
feature_extractor

ViTFeatureExtractor {
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.5,
    0.5,
    0.5
  ],
  "image_processor_type": "ViTFeatureExtractor",
  "image_std": [
    0.5,
    0.5,
    0.5
  ],
  "resample": 2,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "height": 125,
    "width": 125
  }
}

In [ ]:
example = feature_extractor(
    get_image(dataset_train['train'][4]['ImageId']),
    return_tensors='pt'
)
example

In [ ]:
example['pixel_values'].shape, feature_extractor.size

In [7]:
train_transforms = transforms.Compose(
        [
            transforms.Resize((125,125)),
            transforms.ToTensor(),
        ]
    )

val_transforms = transforms.Compose(
        [
            transforms.Resize((125,125)),
            transforms.ToTensor(),
        ]
    )

def preprocess_train(example_batch):
    example_batch["pixel_values"] = [
        train_transforms(image) for image in example_batch["X"]
    ]
    return example_batch
  
def preprocess_val(example_batch):
    example_batch["pixel_values"] = [
        val_transforms(image) for image in example_batch["X"]
    ]
    return example_batch

In [ ]:
splits = dataset_train["train"].train_test_split(test_size=0.1, shuffle=True)
#train_ds = dataset_train["train"]
train_ds=splits['train']
val_ds=splits['test']

In [ ]:
train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

In [ ]:
#model_checkpoint = "microsoft/beit-base-patch16-224" 
model_checkpoint = "microsoft/swin-tiny-patch4-window7-224" 
model =  AutoModelForImageClassification.from_pretrained(
    model_checkpoint, 
    num_labels=1, # for regression
    image_size=125,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)

In [ ]:
torch.cuda.is_available()

In [ ]:
model=model.to('cuda')

In [ ]:
batch_size=4

In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-eurosat2",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=30,
    warmup_ratio=0.1,
    logging_steps=50,
    #load_best_model_at_end=True,
    #metric_for_best_model="rmse",
)

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    rmse = mean_squared_error(labels, logits, squared=False)
    return {"rmse": rmse}

In [ ]:
import torch

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [ ]:
def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["Target"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [ ]:
trainer = RegressionTrainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,  
)

In [ ]:
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
train_results = trainer.train()

In [ ]:
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state() 

In [ ]:
trainer.eval_dataset=val_ds
trainer.evaluate()